In [1]:
import pandas as pd
import sqlite3
db = sqlite3.connect('claims.db')

In [1]:
# This was to populate p1-15, d1-25 the cleaner and lazier way

def proc_diag(s,start, end): #this function is to help me populate the columns D1-D25, and P1-P15
    result = []

    for i in range(start, end+1):
        result.append(s + str(i))
    
    return result


p = proc_diag('P',1,15)
d = proc_diag('D',1,25)


def concat_columns(lst):
    result = ''
    for i in lst:
        result += str(i) + '||' 
    return result[:-2]


d_columns = concat_columns(d)

# this is to check every combo of p1-p15 and d1-d25
def long_condition(proc_diag, code_number): # can only use on specific diagnoses
    result = ""
    for i in proc_diag:
        for j in code_number:
            result += str(i)+ ' = \'' + str(j)+ '\'' + ' OR '
    return result

check_for_maglignant_colon = long_condition(d, ['152'])


#changes list to string
def list_to_string (lst):
    result = ''
    for i in lst:
        result += str(i) + ', ' 
    return result[:-2]

d_string = list_to_string(d)


pcs = [45.8,45.7,45.23]    
p_diagnose_code = long_condition(p, pcs) 




In [167]:
# this code works well if you only are checking for one item and it's a little more general
# in our assignment it says 152. and I could hard code 152.1, 152.2 but I wanted to try it a different way

def diagnosis (diagnosis_code) : 
    return pd.read_sql_query(
                            'SELECT encounter_key, patient_id, doctor_id, ' + d_columns + ''' as diagnoses
                              FROM medical_headers
                              WHERE diagnoses LIKE '%'''
                                + diagnosis_code + "%'",db)

In [168]:
#These people needed colonoscopies
colon_cancer = diagnosis('152.')

In [3]:
def multiple_diagnosis (list_diagnosis_code):
    return pd.read_sql_query('''SELECT encounter_key, patient_id, doctor_id
                               FROM medical_headers
                              WHERE ''' + list_diagnosis_code [:-3] + ''' and patient_id 
                              NOT IN (SELECT patient_id
                              FROM medical_headers
                              WHERE ''' + d_columns + ''' LIKE '%152%')
                              ''', db)


# def multiple_diagnosis (list_diagnosis_code):
#     return pd.read_sql_query('''SELECT encounter_key, patient_id, doctor_id
#                                FROM medical_headers
#                               WHERE ''' + list_diagnosis_code [:-3] + ''' and patient_id 
#                               NOT IN''' colon_cancer.patient_id
# This would be the better way to do it. check for the patient_id's in the magliment dataframe
# It the patient_id is there than they cannot be in this dataframe
  

In [170]:
begnin = long_condition(d, ['211.3', '211.4'])
#These people do NOT need colonoscopies this is my denomonator
not_colon_cancer = multiple_diagnosis(begnin)

In [177]:
not_colon_cancer.info()    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9699 entries, 0 to 9698
Data columns (total 3 columns):
encounter_key    9699 non-null object
patient_id       9699 non-null object
doctor_id        9699 non-null object
dtypes: object(3)
memory usage: 227.4+ KB


In [194]:

def nums_between (start,end):
    result = []
    for i in range (start, end+1):
        result.append(i)
        
    return result

cpt = [45378, 45388, 44110 ] + nums_between(45380,45385) + nums_between (44150,44160) + nums_between(44204, 44208) + nums_between(44210, 44212) + [44146]
only_resection = [44110,44146] + nums_between (44150,44160) +nums_between(44204, 44208) + nums_between(44210, 44212)

In [195]:
cpt_string = list_to_string(cpt)
resection_string = list_to_string(only_resection)

In [184]:
cpt_string

'45378, 45388, 44110, 45380, 45381, 45382, 45383, 45384, 45385, 44150, 44151, 44152, 44153, 44154, 44155, 44156, 44157, 44158, 44159, 44160, 44204, 44205, 44206, 44207, 44208, 44210, 44211, 44212, 44146'

In [187]:
#Colonoscopy/colectomy was preformed, this is my numerator

cc = pd.read_sql_query('''SELECT encounter_key, procedure
                               FROM medical_service_lines
                              WHERE procedure in (''' + cpt_string + ''')''', db)


In [196]:
#This is for only those who got a colectomy

resection = pd.read_sql_query('''SELECT encounter_key, procedure
                               FROM medical_service_lines
                              WHERE procedure in (''' + resection_string+ ''')''', db)


In [4]:

fp= not_colon_cancer.merge(cc, left_on = not_colon_cancer.encounter_key, right_on= cc.encounter_key)
fp.drop(['encounter_key_x', 'encounter_key_y'], axis = 1)

print()

NameError: name 'not_colon_cancer' is not defined

In [191]:
result = pd.merge(not_colon_cancer, cc, how="left", on=["encounter_key"])


In [202]:
ben_got_col =  pd.merge(not_colon_cancer, resection, how="left", on=["encounter_key"])

KeyError: 'patient_id'

In [208]:
# not_colon_cancer['patients'] = not_colon_cancer.groupby(['doctor_id'])
#not enough time but i was going to use the groupby from pandas library to gather how many patients the doctor had
#how many were bengin without any procedures and how many had procedures despite being bengin

In [ ]:
pd.read_sql_query('''SELECT encounter_key, patient_id, doctor_id
                               FROM medical_headers
                              WHERE ''' + list_diagnosis_code [:-3] + ''' and patient_id 
                              NOT IN (SELECT patient_id
                              FROM medical_headers
                              WHERE ''' + d_columns + ''' LIKE '%152%')
                              ''', db)

In [2]:
#easy way to list all the D1-25 and P1-15
def proc_diag(s,start, end): #this function is to help me populate the columns D1-D25, and P1-P15
    result = []

    for i in range(start, end+1):
        result.append(s + str(i))
    
    return result


p = proc_diag('P',1,15)
d = proc_diag('D',1,25)

In [3]:
        for i in range(1, 26):
            self.d.append('D' + str(i))

['P1',
 'P2',
 'P3',
 'P4',
 'P5',
 'P6',
 'P7',
 'P8',
 'P9',
 'P10',
 'P11',
 'P12',
 'P13',
 'P14',
 'P15']

In [ ]:
pd.read_sql_query('''SELECT encounter_key, patient_id, doctor_id
                               FROM medical_headers
                              WHERE ''' + list_diagnosis_code [:-3] + ''' and patient_id 
                              NOT IN (SELECT patient_id
                              FROM medical_headers
                              WHERE ''' + d_columns + ''' LIKE '%152%')
                              ''', db)